## This notebook serves a testing labs for the final implemented code in the .py scripts.

In [ ]:
# !pip install sentence_transformers
# !pip install transformers

In [1]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline

llm_name = "EleutherAI/pythia-160m-deduped" # main language model
sam_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # model for measuring sentiment
ep_name = "cardiffnlp/twitter-roberta-base-emoji"  # model for emoji prediction

generator = pipeline('text-generation',
                     model=llm_name)

sentiment_analysis = pipeline("sentiment-analysis",
                              model=ep_name) # for measuring sentiment

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [2]:
prompt = "happy"

batch_size = 2
max_tokens = 128
temp = 2.0

In [3]:
def generate_response(prompt):
    master_prompt = "Q: tired. A: I'm so exhausted. I feel like I carry a huge weight on my shoulders. Q: happy. A: I am filled to the brim with jubilation right now! I feel like I'm on the moon! Q: I'm annoyed. A: Ugh, this is really ticking me off! Q: {p}. A:".format(p=prompt)
    generator.tokenizer.pad_token_id = generator.model.config.eos_token_id

    output = generator(master_prompt,
            batch_size = 64,
            do_sample = True,
            max_length = 128,
            temperature = 2.0,
            return_full_text = True)

    output = [x["generated_text"] for x in output]
    output = output[0].split("\n")

    answer = output[0].find("Q: {p}. A:".format(p=prompt))
    

    return output[0][answer + len("Q: {p}. A:".format(p=prompt)):]

In [6]:
generate_response(prompt="I feel great")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


" Hitting every note on your phone at the same time every evening can suck, so you better get those at the night's best. B: Sitting through lunch tonight made my bed last thing night. C: My car fell! P:"

In [9]:
def generate_next_word(word):
    generator.tokenizer.pad_token_id = generator.model.config.eos_token_id

    output = generator([word]*5,
            batch_size = 64,
            do_sample = True,
            max_length = 5,
            temperature = 3.0,
            return_full_text = True)

    output = [x[0]["generated_text"] for x in output]
 
    return [output[i].split()[-1] for i in range(len(output)) ]

In [14]:
generate_next_word("we")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['exact', 'on', 'few', 'or', 'the']

In [15]:
next_word = "I"
prompt = "Without using contractions, guess a word that might come after this one: " + next_word

sentence_list = generate_next_word(prompt)
sentence_list

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/Users/jeffreywilliams/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1362: UserWarning: Input length of input_ids is 15, but `max_length` is set to 5. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


['may', 'can', 'know', "I'll", 'couldn']

In [ ]:
def get_tts(text_prompt):
    audio_array = generate_audio(text_prompt)
    return audio_array

In [ ]:
output = generate_response(prompt)

In [ ]:
output

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write
output_tts = get_tts(output)

In [ ]:
save_audio(output_tts)